In [1]:
import config as cfg
import pandas as pd 
import geopandas as gpd
# for plotting
import seaborn as sns
import matplotlib.pyplot as plt
import contextily as ctx
import matplotlib.pyplot as plt
import mapclassify

mobility_data = cfg.MOBILITY_DATA
geometria_data = cfg.GEOMETRIA_DATA
zonificacion_data = cfg.ZONIFICACION_DATA

In [2]:
nombres_distritos = pd.read_csv(zonificacion_data / 'distritos/PROCESSED_nombres_distritos.csv')
gdf = gpd.read_file(zonificacion_data / 'distritos/madrid_gdf.geojson')
renta = pd.read_csv('data/raw/indicadores_renta_2_2021.csv', sep=';') # SOURCE = https://ine.es/jaxiT3/Datos.htm?t=31097

In [3]:
renta['Indicadores de renta media y mediana'].value_counts() # variables of interest

Indicadores de renta media y mediana
Renta neta media por persona                 21
Renta neta media por hogar                   21
Media de la renta por unidad de consumo      21
Mediana de la renta por unidad de consumo    21
Renta bruta media por persona                21
Renta bruta media por hogar                  21
Name: count, dtype: int64

In [4]:
renta['ID'] = renta['Distritos'].str[:7]
renta.drop(columns=['Distritos', 'Municipios', 'Secciones'], inplace=True)

In [5]:
renta_pivot = renta.pivot(index='ID', columns='Indicadores de renta media y mediana', values='Total').reset_index()
renta_pivot.columns.name = None  # Remove the column grouping label
renta_pivot['ID'] = renta_pivot['ID'].astype('int64')
gdf['ID'] = gdf['ID'].astype('int64')

In [6]:
merged = pd.merge(renta_pivot, gdf, on='ID', how='inner') # merging the income and geography data into one gdf
merged = gpd.GeoDataFrame(merged, geometry='geometry')
merged = merged.set_crs(gdf.crs)

In [7]:
merged.to_file("data/processed/geometries_and_income.geojson", driver="GeoJSON")